In [1]:
import numpy as np

In [2]:
bin = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(jobs)] ) )
def calc_cost_per_state(problem_table, state):
    in_jobs = bin(state)
    in_jobs=in_jobs[::-1]
    cost = 0
    for idx, job in enumerate(in_jobs):
        if int(job):
            cost += problem_table[1, idx]
    return cost

In [3]:
def calc_p(problem_table, policy):
    P = np.zeros([states, states])
    for row in range(states):
        for col in range(states):
            curr_state = row
            next_state = col
            if curr_state > 0:
                possible_next_state = curr_state - 2 ** (policy[curr_state] - 1)
            if curr_state == 0:
                if next_state == 0:
                    P[row,col] = 1
                else:
                    P[row,col] = 0
                continue
            if next_state == possible_next_state:
                P[row,col] = problem_table[0, policy[curr_state] - 1]
            elif next_state == curr_state:
                P[row,col] = 1 - problem_table[0, policy[curr_state] - 1]
            else:
                P[row,col] = 0
    return P
            

In [42]:
def get_v(p, r):
    return np.linalg.inv(np.identity(len(r))-0.9*p) * r

def get_v_iteration(p, r):
    v = np.ones([len(r)])
    v_new = np.zeros([len(r)])
    while not (v == v_new).all():
        v = v_new.copy()
        v_new = r + p @ v
    return v

In [49]:
jobs = 5
states = 2 ** jobs
cost = np.zeros([states])
problem_table = np.array([
    [0.6, 0.5, 0.3, 0.7, 0.1],
    [1,     4,   6,   2,   9]
])

for state in range(states):
    cost[state] = calc_cost_per_state(problem_table=problem_table, state=state)
    
policy = np.array([
    0, 1, 2, 1, 3, 1, 3, 1,  # 
    4, 1, 4, 1, 4, 1, 4, 1,  # 
    5, 1, 5, 1, 5, 1, 5, 1,  #
    5, 1, 5, 1, 5, 1, 5, 1   # 
])
p = calc_p(problem_table, policy)


In [48]:
get_v_iteration(p,cost)

array([  0.        ,  10.        ,   8.        ,  58.        ,
        20.        ,  90.        ,  41.33333333, 151.33333333,
         2.85714286,  32.85714286,  16.57142857,  86.57142857,
        31.42857143, 121.42857143,  58.47619048, 188.47619048,
        90.        , 190.        , 138.        , 278.        ,
       170.        , 330.        , 231.33333333, 431.33333333,
       112.85714286, 232.85714286, 166.57142857, 326.57142857,
       201.42857143, 381.42857143, 268.47619048, 488.47619048])

In [46]:
np.identity(len(cost))-p

array([[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [-1. ,  1. ,  0. , ...,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0.5, ...,  0. ,  0. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ...,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0.1,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. , -1. ,  1. ]])